In [68]:
from trie import Trie
from typing import List

products = [
    "apple",
    "application",
    "appetizer",
    "banana",
    "band",
    "banner",
    "ball",
    "bat",
    "battery",
]

class ProductsTrie(Trie):
    def __init__(self, keys = None):
        super().__init__()
        if keys != None:
            self.put_many(keys)
            
    def put_many(self, keys: List[str]):
        for index, key in enumerate(keys):
            self.put(key, index)
    
    def _validate_str(self, text):
        if not isinstance(text, str):
            raise TypeError(f"Illegal argument for countWordsWithPrefix: prefix = {text} must be a string")
        
    def _validate_str_nonempty(self, text):
        if not isinstance(text, str) or not text:
            raise TypeError(f"Illegal argument for contains: key = {text} must be a non-empty string")

In [75]:
class AutocompleteTrie(ProductsTrie):
    def autocomplete(self, prefix):
        return self.keys_with_prefix(prefix)

if __name__ == "__main__":
    trie = AutocompleteTrie(products)

    # user_input = "app"
    user_input = input("What are you searching for? ")
    
    suggestions = trie.autocomplete(user_input)
    print(f"Products search result for '{user_input}':")
    print(suggestions)

Products search result for 'ba':
['banana', 'band', 'banner', 'ball', 'bat', 'battery']


In [74]:
class CountWordsTrie(ProductsTrie):
    def count_words_with_prefix(self, prefix):
        self._validate_str(prefix)

        current = self.root
        for char in prefix:
            if char not in current.children:
                return 0
            current = current.children[char]

        return self._count_words(current)

    def _count_words(self, node):
        count = 1 if node.value is not None else 0
        for child in node.children.values():
            count += self._count_words(child)
        return count


if __name__ == "__main__":
    trie = CountWordsTrie(products)

    # user_input = "app"
    user_input = input("What is the prefix for the words you wish to count? ")

    count = trie.count_words_with_prefix(user_input)
    print(f"Words amount prefixed with '{user_input}': {count}")

Words amount prefixed with 'ba': 6


In [76]:

class T9(ProductsTrie):
    def check_spelling(self, word):
        self._validate_str_nonempty(word)
        return trie.get(word) is not None


if __name__ == "__main__":
    trie = T9(products)

    words_to_check = ["apple", "app", "banner", "bat", "batman"]

    for word in words_to_check:
        if trie.check_spelling(word):
            print(f"'{word}' is written correctly.")
        else:
            print(f"'{word}' not found in the dictionary.")

'apple' is written correctly.
'app' not found in the dictionary.
'banner' is written correctly.
'bat' is written correctly.
'batman' not found in the dictionary.


In [87]:
from collections import deque

class LevenshteinTrie(ProductsTrie):
    def get_corrections(self, word, max_distance=1):
        queue = deque([(self.root, "", 0)])
        corrections = []

        while queue:
            current_node, current_word, current_distance = queue.popleft()

            if current_distance > max_distance:
                continue

            if current_node.value is not None and current_distance > 0:
                corrections.append(current_word)

            for char, next_node in current_node.children.items():
                next_distance = current_distance + (
                    0 if char == word[len(current_word)] else 1
                )
                queue.append((next_node, current_word + char, next_distance))

        return corrections


if __name__ == "__main__":
    trie = LevenshteinTrie(products)

    # word_with_typo = "battary"
    word_with_typo = input("Wat are youuuu loooookin foooorzz? ")

    corrections = trie.get_corrections(word_with_typo, max_distance=2)
    print(f"Possible corrections for '{word_with_typo}':")
    print(corrections)

Possible corrections for 'allle':
['ball', 'apple']
